# Live 29 - Métricas de Classificação

link: https://bit.ly/mario-live29

"regra do 80/20", o Princípio de Pareto : 80% dos eventos (casos comuns) são causados por 20%. 

Aplicando a questâo de estudo métricas: Não há necessiadade de saber de có todas as métricas, pois, sabendo 20% delas você vai ser capaz de aplicals em 80% dos casos.

Vamos ver as métricas que vão resolver a maioria dos problemas

**Obs**
+ No business world, podemos usar métricas difenrete, pois uma classificaçâo certa ou errada pode ter impcato diferente na açâo que ela causa para a empresa.

In [ ]:
import pandas as pd
import numpy as np

Vamos fazer duas classificção a binária e a multivariável

In [ ]:
# seed para reprodutibilidade do experimento
np.random.seed(50)

# Classificação Binária
# ŷ
p_binary = np.random.uniform(size=10) #mdl.predict_proba
# y
y_binary = (np.random.uniform(size=10) > 0.5).astype(int)

# Classificaçâo MultiClasse
# ŷ
p_multi = np.random.uniform(size=(10,3)) #
p_multi_argmax = p_multi.argmax(axis=1)
# y
y_multi = np.random.uniform(size=(10, 3)).argmax(axis=1)

# threshold = ponto de corte; A classificaçâo rece uma prob de [0,1] 
# O ponto de corte por default do skilearn é 0.5
p_binary_threshold = (p_binary > 0.5).astype(int)

## Acurácia
- Definiçâo de Acurácia: **Porcentagme de acerto do modelo**
- não use "oficialmente" (como métrica final a apresentar), apenas "preguiçosamente", há coisas muito melhores
- inadequada para dados desequilibrados, pode te enganar
- Exemplo: Imagine que você vai fazer um detector de spam. Na sua caixa de email hoje, cerca de 98% dos seus emails nâo são span. Por causa disso, se você simplismente atribuir todos os emails como não-spam, você consegue uma acurácia monstruosa de 98% sem ser capaz de detectar um único spam. Isso acaontece porque a qtd de spam é extremamente baixa em realaçâo a qtd de não-spam, ou seja, seu dataset está desbalanceado.

In [ ]:
# Calculando acurácia
from sklearn.metrics import accuracy_score

print("P = {}\nY = {}".format(p_binary_threshold, y_binary))

accuracy_score(y_binary, p_binary_threshold) # acertamos 8/10 = 80%

P = [0 0 0 0 0 1 0 1 1 0]
Y = [0 0 0 1 1 1 0 1 1 0]


0.8

# Precision
- Definição de Precisão: **Dos casos que eu previ como positivos (para uma classe) quantos realmente são?**
- Envio de cupons de desconto, custos diferentes para cada erro.
- Ex: se custa caro mandar a promoção, das pessoas que eu previ que iam comprar, quantas compraram?

https://en.wikipedia.org/wiki/Confusion_matrix


In [ ]:
from sklearn.metrics import precision_score
print("P = {}\nY = {}".format(p_binary_threshold, y_binary))

precision_score(y_binary, p_binary_threshold) # dos casos em que previ 1, acertei todos

# Significado: QUando ele diz que épostivo, ele tem 100% de chance de acertar

P = [0 0 0 0 0 1 0 1 1 0]
Y = [0 0 0 1 1 1 0 1 1 0]


1.0

# Recall
- Definição de Recall: dos que eram realmente positivos (para uma classe) quantos eu detectei?
- Chamado de taxa de detecção

In [ ]:
from sklearn.metrics import recall_score
print("P = {}\nY = {}".format(p_binary_threshold, y_binary))

recall_score(y_binary, p_binary_threshold) # eram 5 positivos, acertei 3 = 60%

# Significa: Detecta 60% dos valores positivos

P = [0 0 0 0 0 1 0 1 1 0]
Y = [0 0 0 1 1 1 0 1 1 0]


0.6

Precisão = 1.0 e Recall = 0.6

Quando diz que é positivos, ele sempre acerta, mas ele só vai detectar para 60% dos registros positivos

# F1 Score
- média harmônica entre os dois  
( 2 * precision * recall ) / (precision + recall) 
https://en.wikipedia.org/wiki/F1_score 
+ Melhor que o Accurácia


In [ ]:
from sklearn.metrics import f1_score

print("P = {}\nY = {}".format(p_binary_threshold, y_binary))
#(2 * .67 * .2857) / (.67 + .2857)

f1_score(y_binary, p_binary_threshold)

P = [0 0 0 0 0 1 0 1 1 0]
Y = [0 0 0 1 1 1 0 1 1 0]


0.7499999999999999

0.4005838652296746

# Kappa
+ Mede a concordância entre seu modelo e um modelo aleartório
+ Uma boa métrica que pouca gente conhece.
+ Costuma-se usar ele em multi-classes
+ Considera-se geralmente uma medida mais robusta do que o simples cálculo percentual de concordância, pois κ leva em consideração a possibilidade da ocorrência de um acaso
- [https://en.wikipedia.org/wiki/Cohen%27s_kappa](https://en.wikipedia.org/wiki/Cohen%27s_kappa)
+ Interpretando
  - Quanto maior o valor kappa, melhor
  - Ele pode ser negativo

In [ ]:
from sklearn.metrics import cohen_kappa_score

print("P = {}\nY = {}".format(p_multi_argmax, y_multi))

cohen_kappa_score(y_multi, p_multi_argmax)

P = [2 1 0 1 2 1 2 0 0 1]
Y = [1 1 1 1 2 2 0 0 1 0]


0.07692307692307687

# Avalia a classificação por probabilidadade diretamente (sem ponto de corte)
+ em sklearn `mdl.predict_proba`
+ Em geral, basta está acima de 50% (ter um ponto de corte) que agente avalia
+ Com essa métricas, verificamos a porcentagem de certeza para da decisão donosso classificador

# Log Loss
- calculada para a probabilidade empírica do evento. Proporção que o evento ocorre na vida real
- Se o time A jogar contra o time B e tiver 40% de chances de ganhar, se jogarem 10 vezes, 4 vezes o time A vai ganhar.
- Se tivermos um modelo para prever isso, entâão, A log loss estará na mínima quando o modelo prever 0.4
- Ou seja, nosso modleo atingir 0.4 significa que está ótimo

**Se um evento no mundo real tem uma probabilidade limitada de acontecer, então nosso modelo também deverá ter essa mesma probabilidade na log loss se for perfeito**

Em um evento onde já se sabe a probaiblidade, já sabemos o limite que uma log loss pode ter, então, quão mais próximo dessa porcentagem melhor o nosos modelo.

A log loss estará minimizada (loss é o erro, erro mínimo == melhor modelo) quando o modelo prever exatamente  a prob de como o evento ocorre na vida real.

**É A MESMA COISA QUE BINARYCROSS ENTROPY = TEORIA DA INFORMAÇÃO**

**EM suma: A log loss é minimizada (modelo perfeito) quando a prob prevista é igual a probabildiade real**

---

Quando usar: A log loss é imporante quando a probabilidade para classificar algo tem que ser bem calibrada.

quanto menor a log losss, melhor

In [ ]:
from sklearn.metrics import log_loss

print("P = {}\nY = {}".format(p_binary, y_binary))

log_loss(y_binary, p_binary)

P = [0.49460165 0.2280831  0.25547392 0.39632991 0.3773151  0.99657423
 0.4081972  0.77189399 0.76053669 0.31000935]
Y = [0 0 0 1 1 1 0 1 1 0]


0.456820673923256

In [ ]:
# Previsâo aleartória
p_random = np.ones(10) * 0.5
log_loss(y_binary, p_random)

# Para uma previsâo binária, seu modelo deve estár abaixo de 0.69

0.6931471805599453

# ROC AUC
- Interpretar ROC-AUC: **Qual é a chance de um exemplo positivo ter um score (previsão) maior do que um negativo?**
- bom quando garantir que positivos sejam rankeados acima dos negativos é mais importante do que prever a probabilidade real do evento
  + Exemplo do Spam: Diferente da log loss,eu não me importa com a probabildiade (a certesa do modelo) em classificar se é span ou não (pois isso depende também no threshold). **Eu quero que o email que tenha mais cara de spam  mesmo seja devidamente classificado como span**

In [ ]:
from sklearn.metrics import roc_auc_score

print("P = {}\nY = {}".format(p_binary, y_binary))

print("ROC Score - Meu Modelo", roc_auc_score(y_binary, p_binary))

print("ROC Score - Modelo Random", roc_auc_score(y_binary, p_random))

P = [0.49460165 0.2280831  0.25547392 0.39632991 0.3773151  0.99657423
 0.4081972  0.77189399 0.76053669 0.31000935]
Y = [0 0 0 1 1 1 0 1 1 0]
ROC Score - Meu Modelo 0.84
ROC Score - Modelo Random 0.5


+ qual é a chance de um exemplo positivo ter um score (previsão) maior do que um negativo?
+ bom quando garantir que positivos sejam rankeados acima dos negativos é mais importante do que prever a probabilidade real do evento

Experimento

+ Suponha que tenha duas caixas, uma com só exemplos positivos e outra com apenas exemplo negativas.
+ Eu quero saber: vou tirar dessas caixas um exemplo positivo e um exemplo de negativo ver a probabildiades do meu modelo e devolver pra caixa (é uma coisa de probabilidade sem reposiçâo, possa pegar o mesmo mais de uma vez)
 - Olho a prob que meu modelo deu para esse exmeplo positivo
 - Olho a prob que meu modelo deu para o exemplo negativo

Se a prob do positivo é maior que negativo, entaô, conto +1.

A porcentagem de veze que o positov > negativo = AUC Score

AUC SCORE = **qual é a chance de um exemplo positivo ter uma prob maior que o do negativo**

É mais interressante quando eu quero saber que os positivos sejam mais identificáveis com certeza que os negativo (de certa forma um pouco relacionado com a Precision para os positivos).

In [ ]:
sum_over = 0
total = 100000

for i in range(total):

  caixa_de_positivos = p_binary[y_binary == 1] # caixa com só positivo
  caixa_de_negativos = p_binary[y_binary == 0] # caixa com só negativo

  positivo = np.random.choice(caixa_de_positivos, size=1, replace=False)
  negativo = np.random.choice(caixa_de_negativos, size=1, replace=False)

  if positivo > negativo:
    sum_over += 1

sum_over / total

0.83897

# AUC da PRC - Area Under the Precision-Recall Curve
- É AVALIAR O ODELO INDEPNDENTE DO THRESHOLD
- acho mais estável e mais fácil de interpretar
- É uma média ponderada da curva de precision/recall
- **VOCÊ CONSEGUE AVALIAR INDEPENDENTE DO PONTO DE CORTE E ALÉM DISSO, VER O DESEMPENHO PARA VÁRIOS PONTOS DE CORTES DIFERENTES**
  + Assim, depois de usála, podemos escolher um ponto de corte bom


[tabela](https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py)

[average_precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html)

In [ ]:
from sklearn.metrics import average_precision_score
print("P = {}\nY = {}".format(p_binary, y_binary))

average_precision_score(y_binary, p_binary)

P = [0.49460165 0.2280831  0.25547392 0.39632991 0.3773151  0.99657423
 0.4081972  0.77189399 0.76053669 0.31000935]
Y = [0 0 0 1 1 1 0 1 1 0]


0.8761904761904762

### Um resuminho de tudo

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_multi, p_multi_argmax))

              precision    recall  f1-score   support

           0       0.33      0.33      0.33         3
           1       0.50      0.40      0.44         5
           2       0.33      0.50      0.40         2

    accuracy                           0.40        10
   macro avg       0.39      0.41      0.39        10
weighted avg       0.42      0.40      0.40        10



## Outros Comentários

=> Uma ideia interressante seria fazer um modelo focado em recall primiero e depois outro apra precision

==> recall x precision: se detectar positivos/negativos for mais importante que acertar positivos/negativos, entâo recall é m ais imporatnte

==> Se nâo há ponto de corte e dados desbalanceados: AUC e PRC 

==> Como saber se ocorreu ovefiting: 
+ Em geral é quando score de testes está diferente do score de treinamento